In [58]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from pymystem3 import Mystem

In [65]:
russian_stops = set(stopwords.words('russian'))
slang_and_others = [u'я', u'а', u'да', u'но', u'тебе', u'мне', u'ты', u'и', u'у', u'на', u'ща', u'ага',
                    u'так', u'там', u'какие', u'который', u'какая', u'туда', u'давай', u'короче', u'кажется', u'вообще',
                    u'ну', u'не', u'чет', u'неа', u'свои', u'наше', u'хотя', u'такое', u'например', u'кароч', u'как-то',
                    u'нам', u'хм', u'всем', u'нет', u'да', u'оно', u'своем', u'про', u'вы', u'м', u'тд',
                    u'вся', u'кто-то', u'что-то', u'вам', u'это', u'эта', u'эти', u'этот', u'прям', u'либо', u'как', u'мы',
                    u'просто', u'блин', u'очень', u'самые', u'твоем', u'ваша', u'кстати', u'вроде', u'типа', u'пока', u'ок',
                   u'ох', u'ах', u'чо', u'шо', u'че', u'аж', u'а', u'б', u'в', u'г', u'д', 
                   u'е', u'ж', u'з', u'и', u'й', u'к', u'л', u'м', u'н', u'о', u'п', u'р', 
                   u'с', u'т', u'у', u'ф', u'х', u'ц', u'ч', u'ш', u'щ', u'э', u'ю', u'я']

for word in slang_and_others:
    russian_stops.add(word)
    
tokenizer = RegexpTokenizer('[а-я]+')

m = Mystem()

In [66]:
def preprocess(text, tokenizer=tokenizer):
    # lower case and remove leading/trailing spaces
    text = text.lower().strip().replace(u'ё', u'е')

    # tokenize the text
    tokens = tokenizer.tokenize(text)
    
    clean_text = ' '.join(tokens)
    
    # apply stemmer to each token
    tokens = (''.join(m.lemmatize(clean_text)[:-1])).split(' ')
    
    tokens = [word for word in tokens if word not in russian_stops]
    
    return tokens

In [67]:
import csv
import hashlib

sentence_hash = set()

csvFile = open('../tweets/ru/all_normalized.csv', 'a')
csvWriter = csv.writer(csvFile)
counter = 0

with open('../tweets/ru/all.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        preprocessed = preprocess(row[1])
        k = hashlib.md5(''.join(preprocessed).encode('utf-8')).hexdigest()
        if k not in sentence_hash:
            sentence_hash.add(k)
            csvWriter.writerow([str(counter)] + preprocessed)
            counter += 1
            